In [1]:
from pyecharts.charts import WordCloud,Pie
import os
import pyecharts.options as opts
import json

# 数据清洗，提取评论内容

In [2]:
# 读取数据
import pandas as pd
train_labled = pd.read_csv('weibo_comments.csv')


In [3]:
train_labled.head(10)

,comments,created_at,gender,location
0,感觉赶车的人要碎了,Fri Jun 27 10:28:12 +0800 2025,m,北京 朝阳区
1,回复@用户5659561968:比我去西站两小时路程好[单身狗][单身狗][单身狗],Fri Jun 27 15:48:45 +0800 2025,f,重庆
2,回复@玛德_最烦装B的人:早晨7点的车，主城区夜里就得起来,Fri Jun 27 13:21:23 +0800 2025,m,其他
3,我这种总爱卡点的完辣[苦涩][苦涩],Fri Jun 27 13:18:31 +0800 2025,f,其他
4,回复@Q顿顿吃不饱:你要进去走的好远,Fri Jun 27 13:16:20 +0800 2025,m,其他
5,没事，有重庆法拉利[太开心],Fri Jun 27 13:07:59 +0800 2025,f,其他
6,这方面杭州西站设计的最好,Fri Jun 27 13:00:04 +0800 2025,m,北京
7,哈哈哈哈哈，我家离东站近，我幸福了[doge][doge][doge],Fri Jun 27 12:51:20 +0800 2025,f,重庆
8,我当年从南京南站的这一头进站跑到另一头检票[微笑],Fri Jun 27 12:07:04 +0800 2025,m,其他
9,你说得对，上次去成都东站比这小好多我都晕了,Fri Jun 27 11:59:04 +0800 2025,f,四川


In [5]:
import re

def clean_comments(text):
    # 如果不是字符串类型，直接返回空字符串
    if not isinstance(text, str):
        return ""
    
    # 处理回复标记
    if text.startswith('回复@'):
        colon_idx = text.find(':')
        if colon_idx != -1:
            text = text[colon_idx + 1:].strip()
    
    # 移除所有表情符号
    text = re.sub(r'\[.*?\]', '', text)
    # 移除所有话题标签
    text = re.sub(r'#.*?#', '', text)
    # 移除URL链接
    text = re.sub(r'http\S+', '', text)
    # 移除连续多余空格
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text



In [6]:
train_labled_qingxi=train_labled

In [7]:
train_labled_qingxi["comments"]=train_labled_qingxi["comments"].apply(lambda x: clean_comments(x))

In [8]:
train_labled_qingxi.head(5)

,comments,created_at,gender,location
0,感觉赶车的人要碎了,Fri Jun 27 10:28:12 +0800 2025,m,北京 朝阳区
1,比我去西站两小时路程好,Fri Jun 27 15:48:45 +0800 2025,f,重庆
2,早晨7点的车，主城区夜里就得起来,Fri Jun 27 13:21:23 +0800 2025,m,其他
3,我这种总爱卡点的完辣,Fri Jun 27 13:18:31 +0800 2025,f,其他
4,你要进去走的好远,Fri Jun 27 13:16:20 +0800 2025,m,其他


In [9]:
train_labled_qingxi.to_csv('weibo_comments_qingshi.csv', index=False)

In [4]:
#将分为三类：0 1 2  #无需判读0 假 1 真2
label = []
ncw_label = train_labled["ncw_label"].tolist()
fake_label = train_labled["fake_label"].tolist()
real_label = train_labled["real_label"].tolist()
for i in range(len(train_labled)):
    if ncw_label[i] == 1 and fake_label[i] == 0 and real_label[i] == 0:
        label.append(0)
    elif ncw_label[i] == 0 and fake_label[i] == 1 and real_label[i] == 0:
        label.append(1)
    elif ncw_label[i] == 0 and fake_label[i] == 0 and real_label[i] == 1:
        label.append(2)
train_labled["label"] = label

In [5]:
print(train_labled.shape)
print(test.shape)
print(train_labled.columns)

(49910, 10)
(2614, 6)
Index(['id', 'content', 'picture_lists', 'category', 'ncw_label', 'fake_label',
       'real_label', 'comment_2c', 'comment_all', 'label'],
      dtype='object')


In [6]:
train_labled.head(3)

,id,content,picture_lists,category,ncw_label,fake_label,real_label,comment_2c,comment_all,label
0,4028762460708675,回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：,NaN,文体娱乐,0,0,1,NaN,NaN,2
1,4126670854660291,//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明.....,63ad082a189566eed7c4bb3e4bc55012.jpg,社会生活,0,0,1,NaN,NaN,2
2,3702441948299962,西宁城管围殴民警扬言要把警察打死|西宁城管围...,4986dc2a5f09a87c7af5dfc57d7775cd.jpg,社会生活,0,0,1,NaN,NaN,2


In [7]:
#多条内容或评论由 t n 分割，
train_labled["content"] = train_labled["content"].astype(str)
train_labled["content"] = train_labled["content"].apply(lambda x: x.replace('\t',''))
train_labled["content"] = train_labled["content"].apply(lambda x: x.replace('\n',''))
train_labled["comment_all"] = train_labled["comment_all"].astype(str)
train_labled["comment_all"] = train_labled["comment_all"].apply(lambda x: x.replace('\t',''))
train_labled["comment_all"] = train_labled["comment_all"].apply(lambda x: x.replace('\n',''))
train_labled["comment_all"] = train_labled["comment_all"].apply(lambda x: x.replace('nan',''))

In [8]:
#train_labled["content"]=train_labled["content"]+train_labled["comment_all"] 
train_labled["content"] = train_labled["content"].apply(lambda x: x.replace(' ',''))

In [9]:
train_labled["content"][1]

'//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明...@网易新闻客户端#网易新闻#'

In [10]:
train_labled.head(3)

,id,content,picture_lists,category,ncw_label,fake_label,real_label,comment_2c,comment_all,label
0,4028762460708675,回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：,NaN,文体娱乐,0,0,1,NaN,,2
1,4126670854660291,//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明.....,63ad082a189566eed7c4bb3e4bc55012.jpg,社会生活,0,0,1,NaN,,2
2,3702441948299962,西宁城管围殴民警扬言要把警察打死|西宁城管围...,4986dc2a5f09a87c7af5dfc57d7775cd.jpg,社会生活,0,0,1,NaN,,2


In [11]:
stats = []
for col in train_labled.columns:
    stats.append((col, train_labled[col].nunique(),train_labled[col].isnull().sum(), train_labled[col].isnull().sum() * 100 / train_labled.shape[0],train_labled[col].dtype))
stats_df = pd.DataFrame(stats, columns=['特征', '值数量','缺失值数量', '缺失值百分比', '类型'])
stats_df.sort_values('缺失值百分比', ascending=False, inplace=True)
stats_df

,特征,值数量,缺失值数量,缺失值百分比,类型
7,comment_2c,21118,27221,54.540172,object
2,picture_lists,28236,20285,40.643158,object
0,id,49910,0,0.000000,object
1,content,49247,0,0.000000,object
3,category,8,0,0.000000,object
4,ncw_label,2,0,0.000000,int64
5,fake_label,2,0,0.000000,int64
6,real_label,2,0,0.000000,int64
8,comment_all,21225,0,0.000000,object
9,label,3,0,0.000000,int64


In [12]:
#由于图片缺失值较多，所以暂时只考虑内容
data_obj=train_labled[['content','comment_all', 'label']]
data_obj.head(3)

,content,comment_all,label
0,回复新浪网友对博文【国家文物局限制鉴宝节目现场估价转】的评论：;;查看原文：,,2
1,//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明.....,,2
2,西宁城管围殴民警扬言要把警察打死|西宁城管围...,,2


In [13]:
print(data_obj['content'][1])

//分享网易新闻:《发生在昆明的火锅店老板“辱滇门”，云南人该愤怒还是羞愧》|发生在昆明...@网易新闻客户端#网易新闻#


In [14]:
#对content进行分词
import jieba
train_labled["content"] = train_labled["content"].apply(lambda x: jieba.lcut(x, cut_all=False))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\16601\AppData\Local\Temp\jieba.cache
Loading model cost 0.648 seconds.
Prefix dict has been built successfully.


In [15]:
train_labled["content"].head(5)

0    [回复, 新浪, 网友, 对, 博文, 【, 国家文物局, 限制, 鉴宝, 节目, 现场, ...
1    [/, /, 分享, 网易, 新闻, :, 《, 发生, 在, 昆明, 的, 火锅店, 老板...
2    [西宁, 城管, 围殴, 民警, 扬言, 要, 把, 警察, 打死, |, 西宁, 城管, ...
3    [【, 川航, 航班, 因, 驾驶舱, 风挡, 破裂, 安全, 备降, 成都, 】, 今天上...
4    [支持, 郑强, ！, ！, ！, /, /, 【, 贵州大学, 校长, 回应, 空姐, 言...
Name: content, dtype: object

In [16]:
##去除停用词
stopword=pd.read_csv('D:\FakeNews/stopwords.txt',encoding="utf-8",quoting=3,names=['stopwords'])
stopword.head(5)

,stopwords
0,日
1,月
2,年
3,时
4,nan


In [17]:
def drop_stopwords(contents,stopwords):
    contents_clean = []
    all_words = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)
            all_words.append(str(word))
        contents_clean.append(line_clean)
    return contents_clean,all_words



In [18]:
contents =train_labled.content.values.tolist()
stopwords = stopword. stopwords.values.tolist()
contents_clean,a1l_words = drop_stopwords(contents,stopwords)


In [19]:
train_labled["content"]=contents_clean

In [20]:
train_labled["content"].head(2)

0    [回复, 新浪, 网友, 博文, 国家文物局, 鉴宝, 节目, 现场, 估价, 转, 评论,...
1    [分享, 网易, 新闻, 发生, 昆明, 火锅店, 老板, 辱, 滇门, 云南, 人该, 愤...
Name: content, dtype: object

In [21]:
#将处理好的表格保存
#train_labled.to_csv('Cleantrain.csv', index=False,encoding='utf-8-sig')

In [22]:
df_allwords=pd.DataFrame({'all_words':a1l_words})
df_allwords.head(3)


,all_words
0,回复
1,新浪
2,网友


In [23]:
#统计词频
import numpy as np
words_count=df_allwords.groupby(by=['all_words' ])['all_words' ].agg({"count": np.size})
words_count=words_count.reset_index().sort_values(by=["count"],ascending=False)
words_count.head(5)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,all_words,count
12553,​,12063
17688,中国,9278
49040,孩子,8705
107620,请,7574
51593,小女孩,6864


In [24]:
words_count.shape

(125304, 2)

In [25]:
word_frequence = {x[0]:x[1] for x in words_count.head(5).values}
word_frequence 

{'\u200b': 12063, '中国': 9278, '孩子': 8705, '请': 7574, '小女孩': 6864}

In [26]:
train_labled["content"][1]

['分享',
 '网易',
 '新闻',
 '发生',
 '昆明',
 '火锅店',
 '老板',
 '辱',
 '滇门',
 '云南',
 '人该',
 '愤怒',
 '羞愧',
 '发生',
 '昆明',
 '网易',
 '新闻',
 '客户端',
 '网易',
 '新闻']

In [27]:
train_labled.head(5)

,id,content,picture_lists,category,ncw_label,fake_label,real_label,comment_2c,comment_all,label
0,4028762460708675,"[回复, 新浪, 网友, 博文, 国家文物局, 鉴宝, 节目, 现场, 估价, 转, 评论,...",NaN,文体娱乐,0,0,1,NaN,,2
1,4126670854660291,"[分享, 网易, 新闻, 发生, 昆明, 火锅店, 老板, 辱, 滇门, 云南, 人该, 愤...",63ad082a189566eed7c4bb3e4bc55012.jpg,社会生活,0,0,1,NaN,,2
2,3702441948299962,"[西宁, 城管, 围殴, 民警, 扬言, 警察, 打死, 西宁, 城管, 围]",4986dc2a5f09a87c7af5dfc57d7775cd.jpg,社会生活,0,0,1,NaN,,2
3,4239549419245466,"[川航, 航班, 驾驶舱, 风挡, 破裂, 备降, 成都, 今天上午, 26, 重庆江北国际...",dcfccfc69e90a0007afd6aafa1385e56.jpg,社会生活,0,0,1,NaN,,2
4,3966337217183260,"[支持, 郑强, 贵州大学, 校长, 回应, 空姐, 言论, 常给, 写, 感谢信]",NaN,社会生活,0,0,1,NaN,,2


In [28]:
groups_label=train_labled.groupby(by=['label' ])['category' ].value_counts()
groups_label=groups_label.reset_index(name='count')
groups_label.head(35)
#print(groups_label)

,label,category,count
0,0,文体娱乐,7612
1,0,科技,2225
2,0,医药健康,1803
3,0,社会生活,1642
4,0,军事,1606
5,0,财经商业,812
6,0,政治,303
7,0,教育考试,101
8,1,社会生活,10203
9,1,医药健康,3321


In [29]:
zunlan_zhonglei=train_labled["category"].value_counts()#总览种类数量
zunlan_zhonglei

社会生活    22258
文体娱乐    10232
医药健康     8123
科技       2506
财经商业     2267
军事       1978
政治       1544
教育考试     1002
Name: category, dtype: int64

In [30]:
zunlan_zhongleiDa=pd.DataFrame(zunlan_zhonglei)
zunlan_zhongleiDa

,category
社会生活,22258
文体娱乐,10232
医药健康,8123
科技,2506
财经商业,2267
军事,1978
政治,1544
教育考试,1002


In [31]:
from pyecharts.charts import Pie#总览种类数量
import pyecharts.options as opts
from pyecharts.globals import ThemeType

pie_zl=Pie()#init_opts=opts.InitOpts(theme=ThemeType.CHALK)
words_zl=[(i,j) for i ,j in zip(zunlan_zhongleiDa.index,zunlan_zhongleiDa["category"].tolist())]
pie_zl.add("新闻种类",words_zl
        
        ,radius=["40%", "85%"]
        ,rosetype='area'
        ,label_opts=opts.LabelOpts(
            position="outside",
            formatter="{d}%"),#,color='#C0C0C0'
            
        )

pie_zl.set_global_opts(title_opts=opts.TitleOpts(title="新闻各类型数量占比", pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18))
                       #title_textstyle_opts=opts.TextStyleOpts(color='#C0C0C0',font_size=23)
                       ,legend_opts=opts.LegendOpts(
     type_="scroll"
     , pos_top="20%"
     , pos_left="80%"#上下边距
     , orient="vertical",textstyle_opts=opts.TextStyleOpts()
     ))   
pie_zl.render("新闻各类型数量占比.html")
pie_zl.render_notebook()


In [32]:
#各个标签下各种类的数量
from pyecharts import options as opts
from pyecharts.charts import Bar
l1=['文体娱乐','科技','医药健康',"社会生活","军事","财经商业","政治","教育考试"]
l2=[7612,2225,1803,1642,1606,812,303,101]
l3=[1267,126,3321,10203,151,693,571,509]
l4=[1353,155,2999,10413,221,762,670,392]
bar_category = (
    Bar(init_opts=opts.InitOpts(width='1000px', height='600px'))#，theme=ThemeType.CHALK
    
    .add_xaxis(l1)
    .add_yaxis("无需判断", l2,category_gap="10%")
    .add_yaxis("假", l3,category_gap="10%")
    .add_yaxis("真", l4,category_gap="10%")
    #.reversal_axis()
    .set_series_opts(
        label_opts=opts.LabelOpts(
            is_show=True,  # 开启标签显示
            position="top"# 标签显示在柱状图上方  ,color='#C0C0C0'
           # formatter="{c}",  # 标签格式化为数据值
        ),
         
    )
    .set_global_opts(title_opts=opts.TitleOpts(title="各类型新闻中的真假分类",pos_bottom="bottom", pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18)),#,color='#C0C0C0'
                     legend_opts=opts.LegendOpts(
     type_="scroll"
     , pos_top="20%"
     , pos_left="80%"#上下边距
     , orient="vertical",textstyle_opts=opts.TextStyleOpts()#color='#C0C0C0'
     ),
     xaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),
            name_textstyle_opts=opts.TextStyleOpts(
                font_family= 'Times New Roman',
                font_size=14,
            ),
#                 坐标轴刻度配置项
            axistick_opts=opts.AxisTickOpts(
#                     is_show=False,  # 是否显示
                is_inside=True,  # 刻度线是否在内侧
            ),
#                 坐标轴线的配置
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(
                    width=1,
                   
                )
            ),
#                 坐标轴标签的配置
            axislabel_opts=opts.LabelOpts(
                font_size=12,
                font_family='Times New Roman',color='black'
            )
        ),
        yaxis_opts=opts.AxisOpts(splitline_opts=opts.SplitLineOpts(is_show=False),
           
            axistick_opts=opts.AxisTickOpts(
                #is_show=, 
                is_inside=True,  # 刻度线是否在内侧
            
            ),
            axislabel_opts=opts.LabelOpts(
                font_size=12,
                font_family='Times New Roman',color='black'
            ),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(
                    width=0.5,
                   
                ))
        ),
                    )
    
)
bar_category.render("新闻各类型中的真假分类.html")
bar_category.render_notebook()

In [33]:
#按分类结果统计词频
lable_0=[]
lable_1=[]
lable_2=[]
for i in range(49910):
    if train_labled['label'][i]==0:
        lable_0=lable_0+train_labled["content"][i]
    if train_labled['label'][i]==1:
        lable_1=lable_1+train_labled["content"][i]
    if train_labled['label'][i]==2:
        lable_2=lable_2+train_labled["content"][i]   
        

In [34]:
print('0:',len(lable_0))
print('1:',len(lable_1))
print('2:',len(lable_2))

0: 580982
1: 614194
2: 669005


In [35]:
#统计词频
result_0 = pd.value_counts(lable_0)
result_0=pd.DataFrame(result_0,columns=['counts'])
result_0.head(5)

,counts
​,12063
说,3746
谣言,3601
中国,2503
真的,2495


In [36]:
result_0.shape

(83885, 1)

In [37]:
result_1 = pd.value_counts(lable_1)
result_1=pd.DataFrame(result_1,columns=['counts'])
result_1.head(5)

,counts
孩子,6795
小女孩,6733
帮忙,5961
请,5881
急,5189


In [38]:
result_1.shape

(29537, 1)

In [39]:
result_2 = pd.value_counts(lable_2)
result_2=pd.DataFrame(result_2,columns=['counts'])
result_2.head(5)

,counts
中国,3589
地震,3084
时,3000
发生,2810
视频,2803


In [40]:
result_2.shape

(63337, 1)

In [41]:
result_0['counts'].values[0:5]

array([12063,  3746,  3601,  2503,  2495], dtype=int64)

In [49]:
def ciyong(name,value,tuming)-> WordCloud: 

    words=[(i,j) for i ,j in zip(name,value.tolist())]
    wordcloud =WordCloud(init_opts=opts.InitOpts(width='450px',height='300px',theme=ThemeType.LIGHT
))#,theme=ThemeType.CHALK 
    wordcloud.add("高频词",words,word_size_range=[20,50], shape='pentagon')#shape词云图轮廓，有'circle', 'cardioid', 'diamond', 'triangle-forward', 'triangle', 'pentagon', 'star'可选
    wordcloud.set_global_opts(title_opts=opts.TitleOpts(title=tuming, pos_left='left',title_textstyle_opts=opts.TextStyleOpts(font_size=23,color='#C0C0C0')),
                      )# 不显示工具箱 ) tooltip_opts=opts.TooltipOpts(is_show=True)
    wordcloud.render(tuming+".html")
    return wordcloud.render_notebook(),wordcloud
    #os.system(tuming+".html")

In [50]:
#无需判断的词云
words=[(i,j) for i ,j in zip(result_0.index[0:20],result_0['counts'].values[0:20].tolist())]
wordcloud_wuxu =WordCloud(init_opts=opts.InitOpts(width='450px',height='300px',theme=ThemeType.LIGHT
))#,theme=ThemeType.CHALK 
wordcloud_wuxu.add("高频词",words,word_size_range=[20,50], shape='pentagon',textstyle_opts=opts.TextStyleOpts(font_family='diamond'))#shape词云图轮廓，有'circle', 'cardioid', 'diamond', 'triangle-forward', 'triangle', 'pentagon', 'star'可选
wordcloud_wuxu.set_global_opts(title_opts=opts.TitleOpts(title="“无需判断”中的高频词", pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
                      )# 不显示工具箱 ) tooltip_opts=opts.TooltipOpts(is_show=True)
wordcloud_wuxu.render("“无需判断”中的高频词.html")
wordcloud_wuxu.render_notebook()
    #os.system(tuming+".html")

In [125]:
#假的词云
words=[(i,j) for i ,j in zip(result_1.index[0:20],result_1['counts'].values[0:20].tolist())]
wordcloud_jia =WordCloud(init_opts=opts.InitOpts(width='450px',height='300px',theme=ThemeType.WHITE
))#,theme=ThemeType.CHALK 
wordcloud_jia.add("高频词",words,word_size_range=[20,50], shape='pentagon')#shape词云图轮廓，有'circle', 'cardioid', 'diamond', 'triangle-forward', 'triangle', 'pentagon', 'star'可选
wordcloud_jia.set_global_opts(title_opts=opts.TitleOpts(title="“假”新闻中的高频词", pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
                      )# 不显示工具箱 ) tooltip_opts=opts.TooltipOpts(is_show=True)
wordcloud_jia.render("“假”新闻中的高频词.html")
wordcloud_jia.render_notebook()

In [126]:
#真的词云
words=[(i,j) for i ,j in zip(result_2.index[0:20],result_2['counts'].values[0:20].tolist())]
wordcloud_zheng =WordCloud(init_opts=opts.InitOpts(width='450px',height='300px',theme=ThemeType.WHITE
))#,theme=ThemeType.CHALK 
wordcloud_zheng.add("高频词",words,word_size_range=[20,50], shape='pentagon',textstyle_opts=opts.TextStyleOpts(font_family='diamond'))#shape词云图轮廓，有'circle', 'cardioid', 'diamond', 'triangle-forward', 'triangle', 'pentagon', 'star'可选
wordcloud_zheng.set_global_opts(title_opts=opts.TitleOpts(title="“真”新闻中的高频词", pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18)),
                      )# 不显示工具箱 ) tooltip_opts=opts.TooltipOpts(is_show=True)
wordcloud_zheng.render("“真”新闻中的高频词.html")
wordcloud_zheng.render_notebook()

In [58]:
buxiangtong_wuxu=[]
buxiangtong_jia=[]
buxiangtong_zhen=[]
for i in result_0.index[0:100]:
    if i not in result_1.index[0:100] and i not in result_1.index[0:100]:
        buxiangtong_wuxu.append(i)
    
    
print('无需判断的词',buxiangtong_wuxu)            
# print('假的词',buxiangtong_jia)      
# print('真的词',buxiangtong_zhen)  

无需判断的词 ['\u200b', '谣言', '展开', '全文', '##', '中', '视频', '流传', '微博', '网络', '做', '链接', '想', '造谣', '希望', '发布', '网传', '网页', '网友', '辟谣', '时', '传言', '美国', '事', '里', '喜欢', '公司', '新', '网上', '回应', '消息', '媒体', '真的假', '事件', '拍', '工作', '时间', '号', '情况', '疯传', '传谣', '相关', '粉丝', '老师', '真相', '秒', '太', '近日', '称', '世界', '国家', '新闻', '前', '\ue627', '事情', '有人', '核实', '关注', '2018', '核查', '北京', '发现', '生活', '传播', '爱', '社会', '电影', '评论', '感觉', '影响', '手机', '照片', '发生', '朋友圈', '听', 'xe627', '工作室', '调查', '经典', '图', '假', '官方', '学生', '&#', '问', '警方']


In [59]:
for i in result_1.index[0:100]:
    if i not in result_0.index[0:100] and i not in result_2.index[0:100]:
        buxiangtong_jia.append(i)
print('假的词',buxiangtong_jia)   

假的词 ['小女孩', '帮忙', '急', '扩散', '大人', '拐走', '抱', '监控', '爸爸', '说出', '男人', '知情者', '疯', '一个四十多岁', '现', '请告', '感谢', '今天上午', '万分', '联系人', '手机号码', '留意', '一个三岁', '小孩', '兄弟姐妹', '小区', '寻人启事', '花园', '锦绣', '转', '万', '爽歪歪', '牛奶', '13940292999', '酬金', '线索', '找', '喝', '可口可乐', '星', '家里', '求', '含有', '宁继春', '13759695559', '告诉', '饮料', '宝宝', '杆菌', '爱心', '张静杰', '肉毒', '妇幼保健', '院', 'AD', '娃哈哈', '添加剂', '钙奶', '粒奶', '源果', '优', '菠萝', '味', '转转', '旺仔', '提示', '美汁', '紧急', '求转', '谢谢', '召回', '多菌灵', '小男孩', '接力', '票房', '小学', '逸夫', '伤', '李', 'QQ']


In [60]:
for i in result_2.index[0:100]:
    if i not in result_1.index[0:100] and i not in result_0.index[0:100]:
        buxiangtong_zhen.append(i)
print('真的词',buxiangtong_zhen)   

真的词 ['地震', '级', '男子', '度', '游客', '现场', '车辆', '分享', '民警', '四川', '正式', '一名', '西安', '记者', '20', '网易', '女子', '动物园', '交警', 'gt', '身边', '司机', '东经', '北纬', '台网', '分在', '测定', '遭', '名', '老虎', '天气', '成都', '15', '30', '13', '14', '女', '医生', '死亡', '深度', '通报', '17', '下午', '平安', '元', '18', '滴滴', '地铁', '震源', '部门', '公安']


In [61]:
# Ci_yong_1,Ciyong_wuxu=ciyong(result_0.index[0:30],result_0['counts'].values[0:30],"无需判断的词云")#无需判断的词云
# Ci_yong_1

In [62]:
# Ci_yong_2,Ciyong_jia=ciyong(result_1.index[0:30],result_1['counts'].values[0:30],"假的词云")#假的词云
# Ci_yong_2

In [63]:
# Ci_yong_3,Ciyong_zheng=ciyong(result_2.index[0:30],result_2['counts'].values[0:30],"真的词云")#真的词云
# Ci_yong_3

In [105]:
def bingtu(attr,v1,names)-> Pie:

    words=[(i,j) for i ,j in zip(attr,v1.tolist())]
    pie=Pie(init_opts=opts.InitOpts(width='450px',height='350px',theme=ThemeType.WHITE))#init_opts=opts.InitOpts(theme=ThemeType.CHALK)
    pie.add("高频词",words
        
        ,radius=["30%", "65%"]
        ,rosetype='area'
        ,label_opts=opts.LabelOpts(is_show=False,
             #position="outside",
             #formatter="{d}%"
                                  ),
            
        )
  #  pie =Pie(names,  width=700,title_text_size=40)
    #pie.add("高频词",attr1,v11,center=[50, 55], is_random=True, radius=[30, 75], rosetype='area', is_legend_show=False, is_label_show=True)
    pie.set_global_opts(title_opts=opts.TitleOpts(title=names, pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=23))
                       ,legend_opts=opts.LegendOpts(is_show=False,
#       type_="scroll"
#      , pos_top="10%"
#      , pos_left="80%",#上下边距   
#       orient="vertical"  
     ))                       
    pie.render(names+".html")
    return pie.render_notebook(),pie
    #os.system(names+".html")

In [52]:
#无需判断的饼图"
words=[(i,j) for i ,j in zip(result_0.index[2:17],result_0['counts'].values[2:17].tolist())]
pie_wuxu=Pie(init_opts=opts.InitOpts(width='450px',height='350px',theme=ThemeType.WHITE))#init_opts=opts.InitOpts(theme=ThemeType.CHALK)
pie_wuxu.add("高频词",words
        
        ,radius=["30%", "65%"]
        ,rosetype='area'
        ,label_opts=opts.LabelOpts(
             position="outside",
             formatter="{d}%"
                                  ),
            
        )
  #  pie =Pie(names,  width=700,title_text_size=40)
    #pie.add("高频词",attr1,v11,center=[50, 55], is_random=True, radius=[30, 75], rosetype='area', is_legend_show=False, is_label_show=True)
pie_wuxu.set_global_opts(title_opts=opts.TitleOpts(title="“无需判断”中的高频词占比",pos_bottom="bottom", pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18))
                       ,legend_opts=opts.LegendOpts(is_show=False,
#       type_="scroll"
#      , pos_top="10%"
#      , pos_left="80%",#上下边距   
#       orient="vertical"  
     ))                       
pie_wuxu.render("“无需判断”中的高频词占比.html")
pie_wuxu.render_notebook()

In [129]:
#假的饼图
words=[(i,j) for i ,j in zip(result_1.index[0:15],result_1['counts'].values[0:15].tolist())]
pie_jia=Pie(init_opts=opts.InitOpts(width='450px',height='350px',theme=ThemeType.WHITE))#init_opts=opts.InitOpts(theme=ThemeType.CHALK)
pie_jia.add("高频词",words
        
        ,radius=["30%", "65%"]
        ,rosetype='area'
        ,label_opts=opts.LabelOpts(
             position="outside",
             formatter="{d}%"
                                  ),
            
        )
  #  pie =Pie(names,  width=700,title_text_size=40)
    #pie.add("高频词",attr1,v11,center=[50, 55], is_random=True, radius=[30, 75], rosetype='area', is_legend_show=False, is_label_show=True)
pie_jia.set_global_opts(title_opts=opts.TitleOpts(title="“假”新闻中的高频词占比", pos_bottom="bottom",pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18))
                       ,legend_opts=opts.LegendOpts(is_show=False,
#       type_="scroll"
#      , pos_top="10%"
#      , pos_left="80%",#上下边距   
#       orient="vertical"  
     ))                       
pie_jia.render("“假”新闻中的高频词占比.html")
pie_jia.render_notebook()

In [130]:
#真的饼图
words=[(i,j) for i ,j in zip(result_2.index[0:15],result_2['counts'].values[0:15].tolist())]
pie_zheng=Pie(init_opts=opts.InitOpts(width='450px',height='350px',theme=ThemeType.WHITE))#init_opts=opts.InitOpts(theme=ThemeType.CHALK)
pie_zheng.add("高频词",words
        
        ,radius=["30%", "65%"]
        ,rosetype='area'
        ,label_opts=opts.LabelOpts(
             position="outside",
             formatter="{d}%"
                                  ),
            
        )
  #  pie =Pie(names,  width=700,title_text_size=40)
    #pie.add("高频词",attr1,v11,center=[50, 55], is_random=True, radius=[30, 75], rosetype='area', is_legend_show=False, is_label_show=True)
pie_zheng.set_global_opts(title_opts=opts.TitleOpts(title="“真”新闻中的高频词占比",pos_bottom="bottom", pos_left='center',title_textstyle_opts=opts.TextStyleOpts(font_size=18))
                       ,legend_opts=opts.LegendOpts(is_show=False,
#       type_="scroll"
#      , pos_top="10%"
#      , pos_left="80%",#上下边距   
#       orient="vertical"  
     ))                       
pie_zheng.render("“真”新闻中的高频词占比.html")
pie_zheng.render_notebook()

In [66]:
# bingtu_1,bingtu_wuxu=bingtu(result_0.index[2:30],result_0['counts'].values[2:30],"无需判断的饼图")#无需判断的饼图
# bingtu_1

In [67]:
# bingtu_2,bingtu_jia=bingtu(result_1.index[0:30],result_1['counts'].values[0:30],"假的饼图")#假的饼图
# bingtu_2

In [68]:
# bingtu_3,bingtu_zheng=bingtu(result_2.index[0:30],result_2['counts'].values[0:30],"真的饼图")#真的饼图
# bingtu_3

In [78]:
test.head(5)

,id,content,picture_lists,category,comment_2,comment_all
0,fb7abd30e64904d4aec80913f39c2a4d,因为带了口罩和害怕唾沫的关系，现在街上即便行人不少也基本只有脚步声，在打印店打印东西，店里同...,NaN,疫情,NaN,NaN
1,33862231893127fa396812b4fa9cc709,台词汇编457（@万善公）两周！/钟南山院士呼吁：解决疫情最快，成本最低的方式就是全中国人民...,NaN,疫情,NaN,NaN
2,0fe350647b75a2729c9b6d5604fa4baa,从武汉撤回的日本人，迎接他们的是每人一台救护车，206人=206台救护车，进行隔离。这就是连...,NaN,疫情,NaN,NaN
3,7de07ad7a1eacff14b0ab88303bde62b,武汉地铁。钟院士的防病毒高招: 各位去医院或其他公共场合之前用淡盐水漱一下咽喉部位，回...,NaN,疫情,NaN,NaN
4,73952631593ee8f33a9b42bd66caaf96,重要的事情说三遍！钟南山院士呼吁： 钟南山院士呼吁： 钟南山院士呼吁：​目...,NaN,疫情,NaN,NaN


In [79]:
stats_1 = []
for col in test.columns:
    stats_1.append((col, test[col].nunique(),test[col].isnull().sum(), test[col].isnull().sum() * 100 / test.shape[0],test[col].dtype))
stats_df1 = pd.DataFrame(stats_1, columns=['特征', '值数量','缺失值数量', '缺失值百分比', '类型'])
stats_df1.sort_values('缺失值百分比', ascending=False, inplace=True)
stats_df1

,特征,值数量,缺失值数量,缺失值百分比,类型
2,picture_lists,408,2203,84.27697,object
4,comment_2,870,1700,65.03443,object
5,comment_all,870,1700,65.03443,object
0,id,2614,0,0.00000,object
1,content,2432,0,0.00000,object
3,category,1,0,0.00000,object


In [80]:
test["content"] = test["content"].astype(str)
test["content"] = test["content"].apply(lambda x: x.replace('\t',''))
test["content"] = test["content"].apply(lambda x: x.replace('\n',''))
test["content"] = test["content"].apply(lambda x: x.replace(' ',''))

In [81]:
#test数据集切分
test["content"] = test["content"].apply(lambda x: jieba.lcut(x, cut_all=False))
contents =test.content.values.tolist()
stopwords = stopword. stopwords.values.tolist()
contents_clean,a1l_words = drop_stopwords(contents,stopwords)
test["content"]=contents_clean

In [82]:
test_cut=a1l_words

In [83]:
test_cut =pd.DataFrame(test_cut, columns=['test词频'])

In [84]:
test_cut2=pd.DataFrame(test_cut['test词频'].value_counts())
test_cut2.head(5)

,test词频
武汉,1896
肺炎,1258
感染,1114
疫情,1024
新型,885


In [85]:
Ci_yong_4,Ciyong_ceshi=ciyong(test_cut2.index[0:300],test_cut2['test词频'].values[0:300],'测试集的词云')#测试集的词云
Ci_yong_4

In [86]:
bingtu_4,bingtu_ceshi=bingtu(test_cut2.index[0:10],test_cut2['test词频'].values[0:10],"测试集的饼图")#测试集的饼图
bingtu_4

In [87]:
test.head(5)

,id,content,picture_lists,category,comment_2,comment_all
0,fb7abd30e64904d4aec80913f39c2a4d,"[口罩, 害怕, 唾沫, 关系, 街上, 行人, 脚步声, 打印, 店, 打印, 东西, 店...",NaN,疫情,NaN,NaN
1,33862231893127fa396812b4fa9cc709,"[台词, 汇编, 457, 万善公, 两周, 钟南山, 院士, 呼吁, 解决, 疫情, 最快...",NaN,疫情,NaN,NaN
2,0fe350647b75a2729c9b6d5604fa4baa,"[武汉, 撤回, 日本, 迎接, 每人, 一台, 救护车, 206, 206, 台, 救护车...",NaN,疫情,NaN,NaN
3,7de07ad7a1eacff14b0ab88303bde62b,"[武汉, 地铁, 钟, 院士, 防病毒, 高招, 医院, 公共场合, 淡盐水, 漱, 咽喉,...",NaN,疫情,NaN,NaN
4,73952631593ee8f33a9b42bd66caaf96,"[事情, 说, 三遍, 钟南山, 院士, 呼吁, 钟南山, 院士, 呼吁, 钟南山, 院士,...",NaN,疫情,NaN,NaN


In [88]:
test.to_csv('Cleantestdata.csv', index=False,encoding='utf-8-sig')

In [89]:
#组合图形
# from pyecharts.charts import  Page
# page = Page(layout=Page.SimplePageLayout)    # 简单布局
#     # 将上面定义好的图添加到 page
# page.add(
#     Ciyong_wuxu,
#     Ciyong_jia,
#     Ciyong_zheng,
#     Ciyong_ceshi,
#     bingtu_wuxu,
#     bingtu_jia,
#     bingtu_zheng,
#     bingtu_ceshi
#     )
# page.render("ZuHetu.html")
# page.render_notebook()




In [90]:
# from pyecharts.charts import  Tab
# tab = Tab()
# tab.add(Ciyong_wuxu,"无需判断的词云")
# tab.add(Ciyong_jia,"假的词语云")
# tab.add(Ciyong_zheng,"真的词云")
# tab.add(Ciyong_ceshi,"测试集的词云")
# tab.add(bingtu_wuxu,"无需判断的饼图")
# tab.add(bingtu_jia,"假的饼图")
# tab.add(bingtu_zheng,"真的饼图")
# tab.add(bingtu_ceshi,"测试集的饼图")

# tab.render("zuhetu_xing.html")
# tab.render_notebook()

In [134]:
#大标题
def big_biaoti(name):
    from pyecharts.components import Table
    table = Table()
    table.add(headers=[name], rows=[], attributes={
        "align": "center",
        "border": False,
        "padding": "2px",
        "style": "background:{}; width:1450px; height:50px; font-size:25px; "#.format('#293441') color:#C0C0C0;
    })
    table.render('大标题.html')
    return table

In [146]:
pie_zl.chart_id="1"
bar_category.chart_id="2"
wordcloud_wuxu.chart_id="3"
wordcloud_jia.chart_id="4"
wordcloud_zheng.chart_id="5"
pie_wuxu.chart_id="6"
pie_jia.chart_id="7"
pie_zheng.chart_id="8"

In [147]:
from pyecharts.charts import Page
# 绘制:整个页面
page = Page(
    page_title="数据可视化——虚假新闻检测",
    layout=Page.DraggablePageLayout,  # 拖拽方式
)
page.add(
    big_biaoti("数据可视化——虚假新闻检测"),
    pie_zl,
    bar_category,
    wordcloud_wuxu,
    wordcloud_jia,
    wordcloud_zheng,    
    pie_wuxu,
    pie_jia,
    pie_zheng,    
)
page.render('大屏_临时.html')  # 执行完毕后,打开临时html并排版,排版完点击Save Config，把json文件放到本目录下
print('生成完毕:大屏_临时.html')

生成完毕:大屏_临时.html


In [148]:
Page.save_resize_html(
    source="大屏_临时.html",
    cfg_file="chart_config.json",
    dest="数据可视化_最终.html"
)

'<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="UTF-8">\n    <title>数据可视化——虚假新闻检测</title>\n                <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/echarts.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/echarts-wordcloud.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/jquery.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/jquery-ui.min.js"></script>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/v5/ResizeSensor.js"></script>\n\n            <link rel="stylesheet"  href="https://assets.pyecharts.org/assets/v5/jquery-ui.css">\n\n</head>\n<body >\n    <style>.box {  } </style>\n        \n    <div class="box">\n                        <style>\n            .fl-table {\n                margin: 20px;\n                border-radius: 5px;\n        

In [117]:
# from bs4 import BeautifulSoup
# from pyecharts.charts import Page
# with open("数据可视化_最终.html", "r+", encoding='utf-8') as html:
#     html_bf = BeautifulSoup(html, 'lxml')
#     body = html_bf.find("body")
#     body["style"] = "background-image: url(D:/桌面/2023暑假作业/虚假新闻检测/background/bg_body.jpg)"  
#     html_new = str(html_bf)
#     html.seek(0, 0)
#     html.truncate()
#     html.write(html_new)